In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Get Product Information from Search Page

In [44]:
def get_data2(no_pages):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    
    alls = []
    for i in range(3, no_pages+3):
        r = requests.get('https://www.amazon.com/s?k=board+games&page='+str(i), headers=headers)
        content = r.content
        soup = BeautifulSoup(content)


        for d in soup.findAll('div', attrs={'class':'sg-col-4-of-24 sg-col-4-of-12 sg-col-4-of-36 s-result-item s-asin sg-col-4-of-28 sg-col-4-of-16 sg-col sg-col-4-of-20 sg-col-4-of-32'}):
            name = d.find('span', attrs={'class':'a-size-base-plus a-color-base a-text-normal'})
            price = d.find('span', attrs={'class':'a-offscreen'})
            rating = d.find('span', attrs={'class':'a-icon-alt'})
            no_of_ratings = d.find('span', attrs={'class':'a-size-base'})
            asin = d['data-asin']

            all1=[]
            
            if asin is not None:
                all1.append(asin)
            else:
                all1.append('-')
            
            if name is not None:
                all1.append(name.text)
            else:
                all1.append("unknown-product")

            if price is not None:
                all1.append(price.text)
            else:
                all1.append('$0')

            if rating is not None:
                all1.append(rating.text)
            else:
                all1.append('-')
                
            if no_of_ratings is not None:
                all1.append(no_of_ratings.text)
            else:
                all1.append('-') 
                


            alls.append(all1)  

    return alls

In [45]:
df = pd.DataFrame(get_data2(2),columns=['asin_id',
                                       'name',
                                       'price',
                                       'rating',
                                       'no_of_ratings'])

In [46]:
df.head(10)

,asin_id,name,price,rating,no_of_ratings
0,B077Z1R28P,Taco Cat Goat Cheese Pizza,$9.99,4.8 out of 5 stars,"8,517"
1,B07JZTBV9C,Taco vs Burrito - The Wildly Popular Surprisin...,$19.99,4.4 out of 5 stars,"2,740"
2,B000809OAO,Ticket To Ride - Europe,$44.48,4.8 out of 5 stars,"4,788"
3,B07L437XBJ,Istanbul: Big Box Board Game,$55.89,4.8 out of 5 stars,86
4,B083YKSNN8,"Hasbro Gaming Mall Madness Game, Talking Elect...",$24.99,4.2 out of 5 stars,114
5,B07SFKCPJ7,Latice Hawaii Strategy Board Game - The Remark...,$34.99,4.7 out of 5 stars,780
6,B074YN5CKM,Sid Meier's Civilization - A New Dawn,$32.95,4.5 out of 5 stars,287
7,B0813WY7V8,The Game Designers,$11.99,4.2 out of 5 stars,2019
8,B08412VYKZ,Ravensburger Disney Jungle Cruise Adventure Ga...,$34.99,4.6 out of 5 stars,487
9,B00GNG0BZE,Fun+1 Toys! Classic Wooden Chess Set - Wooden ...,$19.49,4.5 out of 5 stars,429


In [47]:
df.to_csv('csv_files/amazon_items2.csv')

In [48]:
## Board Games on Amazon (First 4 Pages of Search Query)

amazon_items1 = pd.read_csv('amazon_items.csv',index_col=0)
amazon_items2 = pd.read_csv('amazon_items2.csv',index_col=0)

amazon_items = pd.concat([amazon_items1, amazon_items2], sort=False)

amazon_items

,asin_id,name,price,rating,no_of_ratings
0,B076HK9H7Z,Sorry! Game,$0,4.7 out of 5 stars,"7,329"
1,B00D7OAOYI,Trouble Game,$0,4.5 out of 5 stars,"7,698"
2,B00C0ULS3G,Battleship Classic Board Game Strategy Game Ag...,$0,4.7 out of 5 stars,"3,976"
3,B08636S37Z,Funko Disney The Haunted Mansion – Call of The...,$24.97,4.4 out of 5 stars,15
4,B00000IVAK,SEQUENCE- Original SEQUENCE Game with Folding ...,$15.17,4.8 out of 5 stars,"11,519"
...,...,...,...,...,...
91,B00UB7P0XY,Gamewright Outfoxed! A Cooperative Whodunit Bo...,$19.99,4.7 out of 5 stars,"1,861"
92,B085WL16TV,Funkoverse: Disney The Nightmare Before Christ...,$39.99,4.9 out of 5 stars,20
93,B087CSGV84,"BAKAM Super Fast Sling Puck Game, Portable Tab...",$39.99,4.5 out of 5 stars,106
94,1933054395,Mindware Qwirkle Board Game,$24.99,4.8 out of 5 stars,"6,784"


In [49]:
amazon_items.to_csv('csv_files/amazon_items_final.csv')

# Get Reviews for each Item

In [50]:
def get_reviews(asin_no, no_review_pages):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    
    alls = []
    for i in range(1, no_review_pages+1):
        r = requests.get('https://www.amazon.com/product-reviews/'+asin_no+'/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber='+str(i), headers=headers)
        content = r.content
        soup = BeautifulSoup(content)

        for d in soup.findAll('div', attrs={'class':'a-section review aok-relative'}):
            product_id = d['id']
            review_title = d.find('a', attrs={'class':'a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold'})
            review_description = d.find('span', attrs={'class':'a-size-base review-text review-text-content'})
            rating = d.find('span', attrs={'class':'a-icon-alt'})
#             user_name = d.find('span', attrs={'class': 'a-profile-name'}).text

            all1=[]
            
            all1.append(asin_no)
            
            if product_id is not None:
                all1.append(product_id)
            else:
                all1.append("unknown-product")

            if review_title is not None:
                all1.append(review_title.text)
            else:
                all1.append('$0')

            if review_description is not None:
                #print(rating.text)
                all1.append(review_description.text)
            else:
                all1.append('-1')

            if rating is not None:
                #print(price.text)
                all1.append(rating.text)
            else:
                all1.append('0')


            alls.append(all1)  

    return alls

In [51]:
# append reviews from all items to list

reviews = []

for id_no in df['asin_id']:
    for row in get_reviews(id_no,5):
        reviews.append(row)

KeyboardInterrupt: 

In [ ]:
reviews

In [ ]:
df_reviews = pd.DataFrame(reviews,columns=['asin_id',
                                       'review_id',
                                       'review_title',
                                       'review_description',
                                       'rating'])

In [ ]:
df_reviews

In [ ]:
df_reviews.to_csv('csv_files/amazon_reviews2.csv')

In [ ]:
amazon_reviews1 = pd.read_csv('amazon_reviews.csv',index_col=0)
amazon_reviews2 = pd.read_csv('amazon_reviews2.csv',index_col=0)

amazon_reviews = pd.concat([amazon_reviews1, amazon_reviews2], sort=False)

amazon_reviews

In [52]:
amazon_reviews.to_csv('csv_files/amazon_reviews_final.csv')